# Table of Contents
## 1.0 Setup
## 2.0 Data Cleaning and Wrangling
## 3.0 Exporting data

## 1.0 Setup

In [87]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import os

In [88]:
# Defining the path

path = (r'E:\OneDrive\ANITA\DATA ANALYST\IMMERSION\Achievement 6\11-22 Food Balance Sheet Analysis\2.0 Data')

In [89]:
# Importing fbs_start.pkl as fbs

fbs = pd.read_pickle(os.path.join(path, '2.1 Original data', 'fbs_start.pkl'))

## 2.0 Data Cleaning and Wrangling

In [90]:
fbs.head()

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code (CPC),Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,FBS,Food Balances (2010-),4,Afghanistan,511,Total Population - Both sexes,S2501,Population,2010,2010,1000 persons,29186.0,X,Figure from international organizations
1,FBS,Food Balances (2010-),4,Afghanistan,511,Total Population - Both sexes,S2501,Population,2011,2011,1000 persons,30117.0,X,Figure from international organizations
2,FBS,Food Balances (2010-),4,Afghanistan,511,Total Population - Both sexes,S2501,Population,2012,2012,1000 persons,31161.0,X,Figure from international organizations
3,FBS,Food Balances (2010-),4,Afghanistan,511,Total Population - Both sexes,S2501,Population,2013,2013,1000 persons,32270.0,X,Figure from international organizations
4,FBS,Food Balances (2010-),4,Afghanistan,511,Total Population - Both sexes,S2501,Population,2014,2014,1000 persons,33371.0,X,Figure from international organizations


In [91]:
# Deleting unnecessary columns

fbs = fbs.drop(columns = ['Domain Code', 'Domain', 'Area Code (M49)', 'Item Code (CPC)', 'Year Code', 'Flag', 'Flag Description'])

In [92]:
# Renaming column 'Area' into 'Country'

fbs.rename(columns = {'Area' : 'Country'}, inplace = True)

In [93]:
fbs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000600 entries, 0 to 200039
Data columns (total 7 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Country       object 
 1   Element Code  int64  
 2   Element       object 
 3   Item          object 
 4   Year          int64  
 5   Unit          object 
 6   Value         float64
dtypes: float64(1), int64(2), object(4)
memory usage: 183.1+ MB


In [94]:
# Checking the countries

fbs['Country'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bangladesh', 'Barbados',
       'Belarus', 'Belgium', 'Belgium-Luxembourg', 'Belize', 'Benin',
       'Bolivia (Plurinational State of)', 'Bosnia and Herzegovina',
       'Botswana', 'Brazil', 'Bulgaria', 'Burkina Faso', 'Burundi',
       'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Comoros', 'Congo', 'Costa Rica', "Côte d'Ivoire", 'Croatia',
       'Cuba', 'Cyprus', 'Czechia',
       "Democratic People's Republic of Korea",
       'Democratic Republic of the Congo', 'Denmark', 'Djibouti',
       'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt',
       'El Salvador', 'Estonia', 'Eswatini', 'Ethiopia', 'Ethiopia PDR',
       'Fiji', 'Finland', 'France', 'French Polynesia', 'Gabon', 'Gambia',
       'Georgia', 'Germany', 'Ghana', 'Greece',

In [95]:
# Creating a list of countries to delete

areas = ['Belgium-Luxembourg', 'Ethiopia PDR', 'Netherlands Antilles (former)', 'Serbia and Montenegro']

In [96]:
# Deleting the countires from the fbs data frame

fbs = fbs[~fbs['Country'].isin(areas)]

In [97]:
# Changing the data type of the column "Element Code"

fbs['Element Code'] = fbs['Element Code'].astype('str')

### Basic food supply variables

In [98]:
# Creating a data frame with basic food supply variables

element_codes = ['511', '5511', '5611', '5911', '5301', '5142']
fbs1 = fbs[fbs['Element Code'].isin(element_codes)]

In [99]:
fbsv = pd.crosstab(index = [fbs1['Country'], fbs1['Year']], columns = fbs1['Element'], values = fbs1['Value'], aggfunc = 'sum').reset_index()

In [100]:
# Renaming column 'Total Population - Both sexes' into 'Total Population'

fbsv.rename(columns = {'Total Population - Both sexes' : 'Total Population'}, inplace = True)

In [101]:
fbsv.head()

Element,Country,Year,Domestic supply quantity,Export Quantity,Food,Import Quantity,Production,Total Population
0,Afghanistan,2010,13164.26,360.0,10075.26,2738.01,11014.25,29186.0
1,Afghanistan,2011,12995.00,276.0,10235.00,3377.00,9462.00,30117.0
2,Afghanistan,2012,13851.78,198.0,10719.78,2953.00,11409.78,31161.0
3,Afghanistan,2013,14541.56,281.0,11233.56,3219.00,11552.56,32270.0
4,Afghanistan,2014,15935.36,408.0,12454.36,3668.00,12938.36,33371.0


In [102]:
fbsv.shape

(1760, 8)

In [103]:
# Creating Import Dependency Ratio variable

fbsv['Import Dependency Ratio'] = (fbsv['Import Quantity'] * 100) / (fbsv['Production'] + fbsv['Import Quantity'] - fbsv['Export Quantity'])

In [104]:
# Creating Self-sufficiency Ratio variable

fbsv['Self-sufficiency Ratio'] = (fbsv['Production'] * 100) / (fbsv['Production'] + fbsv['Import Quantity'] - fbsv['Export Quantity'])

In [105]:
# Creating Population, milions variable

fbsv['Population, milions'] = fbsv['Total Population']/1000

In [106]:
# Creating Production, tonnes variable

fbsv['Production, kg'] = fbsv['Production']*1000*1000

In [107]:
fbsv.head()

Element,Country,Year,Domestic supply quantity,Export Quantity,Food,Import Quantity,Production,Total Population,Import Dependency Ratio,Self-sufficiency Ratio,"Population, milions","Production, kg"
0,Afghanistan,2010,13164.26,360.0,10075.26,2738.01,11014.25,29186.0,20.444720,82.243400,29.186,1.101425e+10
1,Afghanistan,2011,12995.00,276.0,10235.00,3377.00,9462.00,30117.0,26.880522,75.316405,30.117,9.462000e+09
2,Afghanistan,2012,13851.78,198.0,10719.78,2953.00,11409.78,31161.0,20.847482,80.550351,31.161,1.140978e+10
3,Afghanistan,2013,14541.56,281.0,11233.56,3219.00,11552.56,32270.0,22.214462,79.724731,32.270,1.155256e+10
4,Afghanistan,2014,15935.36,408.0,12454.36,3668.00,12938.36,33371.0,22.644268,79.874506,33.371,1.293836e+10


### Basic food availability variables

In [108]:
# Creating a data frame with basic food availability variables

element_codes = ['645', '664', '674', '684']
fbs2 = fbs[fbs['Element Code'].isin(element_codes)]

In [109]:
fbs2

,Country,Element Code,Element,Item,Year,Unit,Value
140,Afghanistan,645,Food supply quantity (kg/capita/yr),Wheat and products,2010,kg,168.72
141,Afghanistan,645,Food supply quantity (kg/capita/yr),Wheat and products,2011,kg,162.49
142,Afghanistan,645,Food supply quantity (kg/capita/yr),Wheat and products,2012,kg,158.01
143,Afghanistan,645,Food supply quantity (kg/capita/yr),Wheat and products,2013,kg,161.62
144,Afghanistan,645,Food supply quantity (kg/capita/yr),Wheat and products,2014,kg,158.61
...,...,...,...,...,...,...,...
200035,Zimbabwe,684,Fat supply quantity (g/capita/day),Miscellaneous,2015,g/capita/day,0.03
200036,Zimbabwe,684,Fat supply quantity (g/capita/day),Miscellaneous,2016,g/capita/day,0.03
200037,Zimbabwe,684,Fat supply quantity (g/capita/day),Miscellaneous,2017,g/capita/day,0.01
200038,Zimbabwe,684,Fat supply quantity (g/capita/day),Miscellaneous,2018,g/capita/day,0.01


In [110]:
fapv = pd.crosstab(index = [fbs2['Country'], fbs2['Year']], columns = fbs2['Element'], values = fbs2['Value'], aggfunc = 'sum').reset_index()

In [111]:
fapv.head()

Element,Country,Year,Fat supply quantity (g/capita/day),Food supply (kcal/capita/day),Food supply quantity (kg/capita/yr),Protein supply quantity (g/capita/day)
0,Afghanistan,2010,36.70,2169.0,345.16,59.21
1,Afghanistan,2011,34.80,2153.0,339.85,58.01
2,Afghanistan,2012,36.50,2158.0,343.92,57.84
3,Afghanistan,2013,37.09,2196.0,348.06,57.72
4,Afghanistan,2014,41.48,2265.0,373.29,60.17


### Basic foods production

In [112]:
# Creating a data frame with production of basic foods

items = ['Wheat and products', 'Rice and products', 'Maize and products', 'Cassava and products',
        'Potatoes and products', 'Beans', 'Soyabeans', 'Bovine Meat', 'Pigmeat', 'Poultry Meat',
        'Eggs']
fbs3 = fbs[fbs['Item'].isin(items)]

In [113]:
fbs3.head()

,Country,Element Code,Element,Item,Year,Unit,Value
10,Afghanistan,5511,Production,Wheat and products,2010,1000 tonnes,4532.0
11,Afghanistan,5511,Production,Wheat and products,2011,1000 tonnes,3388.0
12,Afghanistan,5511,Production,Wheat and products,2012,1000 tonnes,5050.0
13,Afghanistan,5511,Production,Wheat and products,2013,1000 tonnes,5169.0
14,Afghanistan,5511,Production,Wheat and products,2014,1000 tonnes,5370.0


In [114]:
fbs3['Element'].unique()

array(['Production', 'Import Quantity', 'Stock Variation',
       'Export Quantity', 'Domestic supply quantity', 'Feed', 'Seed',
       'Losses', 'Processing', 'Other uses (non-food)',
       'Tourist consumption', 'Residuals', 'Food',
       'Food supply quantity (kg/capita/yr)',
       'Food supply (kcal/capita/day)',
       'Protein supply quantity (g/capita/day)',
       'Fat supply quantity (g/capita/day)'], dtype=object)

In [115]:
# Creating a list of rows which don't contain Production data

prod = ['Import Quantity', 'Stock Variation',
       'Export Quantity', 'Domestic supply quantity', 'Feed', 'Seed',
       'Losses', 'Processing', 'Other uses (non-food)',
       'Tourist consumption', 'Residuals', 'Food',
       'Food supply quantity (kg/capita/yr)',
       'Food supply (kcal/capita/day)',
       'Protein supply quantity (g/capita/day)',
       'Fat supply quantity (g/capita/day)']

In [116]:
# Deleting the rows from the data frame

bfpv = fbs3[~fbs3['Element'].isin(prod)]

In [117]:
bfpv['Element'].unique()

array(['Production'], dtype=object)

In [118]:
# Creating a data frame with production of basic foods

bfpv = pd.crosstab(index = [bfpv['Country'], bfpv['Year']], columns = bfpv['Item'], values = bfpv['Value'], aggfunc = 'sum').reset_index()

In [119]:
bfpv.head(10)

Item,Country,Year,Beans,Bovine Meat,Cassava and products,Eggs,Maize and products,Pigmeat,Potatoes and products,Poultry Meat,Rice and products,Soyabeans,Wheat and products
0,Afghanistan,2010,0.0,131.0,0.0,16.0,301.0,0.0,246.0,28.0,672.0,0.0,4532.0
1,Afghanistan,2011,0.0,138.0,0.0,17.0,300.0,0.0,205.0,26.0,672.0,0.0,3388.0
2,Afghanistan,2012,0.0,139.0,0.0,18.0,310.0,0.0,230.0,25.0,500.0,0.0,5050.0
3,Afghanistan,2013,0.0,134.0,0.0,18.0,312.0,0.0,303.0,26.0,512.0,0.0,5169.0
4,Afghanistan,2014,0.0,121.0,0.0,19.0,316.0,0.0,340.0,25.0,537.0,0.0,5370.0
5,Afghanistan,2015,0.0,111.0,0.0,20.0,316.0,0.0,328.0,25.0,410.0,0.0,4673.0
6,Afghanistan,2016,0.0,106.0,0.0,20.0,312.0,0.0,428.0,24.0,357.0,0.0,4555.0
7,Afghanistan,2017,0.0,95.0,0.0,20.0,174.0,0.0,513.0,28.0,338.0,0.0,4281.0
8,Afghanistan,2018,0.0,92.0,0.0,22.0,107.0,0.0,616.0,29.0,352.0,0.0,3613.0
9,Afghanistan,2019,0.0,95.0,0.0,22.0,185.0,0.0,921.0,28.0,383.0,0.0,4890.0


### Production of essential foods

In [120]:
fbs['Item'].unique()

array(['Population', 'Wheat and products', 'Rice and products',
       'Barley and products', 'Maize and products', 'Rye and products',
       'Oats', 'Millet and products', 'Sorghum and products',
       'Cereals, Other', 'Cassava and products', 'Potatoes and products',
       'Sweet potatoes', 'Yams', 'Roots, Other', 'Sugar cane',
       'Sugar beet', 'Sugar non-centrifugal', 'Sugar (Raw Equivalent)',
       'Sweeteners, Other', 'Honey', 'Beans', 'Peas',
       'Pulses, Other and products', 'Nuts and products', 'Soyabeans',
       'Groundnuts', 'Sunflower seed', 'Rape and Mustardseed',
       'Cottonseed', 'Coconuts - Incl Copra', 'Sesame seed',
       'Palm kernels', 'Olives (including preserved)', 'Oilcrops, Other',
       'Soyabean Oil', 'Groundnut Oil', 'Sunflowerseed Oil',
       'Rape and Mustard Oil', 'Cottonseed Oil', 'Palmkernel Oil',
       'Palm Oil', 'Coconut Oil', 'Sesameseed Oil', 'Olive Oil',
       'Ricebran Oil', 'Maize Germ Oil', 'Oilcrops Oil, Other',
       'Tomat

In [121]:
# Creating a data frame with essential foods

essentials = ['Coffee and products', 'Wine', 'Beer', 'Tea (including mate)']
fbs4 = fbs[fbs['Item'].isin(essentials)]

In [122]:
fbs4['Element'].unique()

array(['Production', 'Import Quantity', 'Stock Variation',
       'Export Quantity', 'Domestic supply quantity', 'Feed', 'Seed',
       'Losses', 'Processing', 'Other uses (non-food)',
       'Tourist consumption', 'Residuals', 'Food',
       'Food supply quantity (kg/capita/yr)',
       'Food supply (kcal/capita/day)',
       'Protein supply quantity (g/capita/day)',
       'Fat supply quantity (g/capita/day)'], dtype=object)

In [123]:
# Deleting the rows from the data frame which do not contain Production data

ess = fbs4[~fbs4['Element'].isin(prod)]

In [124]:
ess['Element'].unique()

array(['Production'], dtype=object)

In [125]:
# Creating a data frame with production of essential foods

essv = pd.crosstab(index = [ess['Country'], ess['Year']], columns = ess['Item'], values = ess['Value'], aggfunc = 'sum').reset_index()

In [126]:
essv.head()

Item,Country,Year,Beer,Coffee and products,Tea (including mate),Wine
0,Afghanistan,2010,0.0,0.0,0.0,0.0
1,Afghanistan,2011,0.0,0.0,0.0,0.0
2,Afghanistan,2012,0.0,0.0,0.0,0.0
3,Afghanistan,2013,0.0,0.0,0.0,0.0
4,Afghanistan,2014,0.0,0.0,0.0,0.0


## 3.0 Exporting data

In [127]:
# Exporting the fbsv data frame to fbsv.csv

fbsv.to_csv(os.path.join(path, '2.2 Prepared data', 'fbsv.csv'))

In [128]:
# Exporting the fapv data frame to fapv.csv

fapv.to_csv(os.path.join(path, '2.2 Prepared data', 'fapv.csv'))

In [129]:
# Exporting the bfpv data frame to bfpv.csv

bfpv.to_csv(os.path.join(path, '2.2 Prepared data', 'bfpv.csv'))

In [131]:
# Exporting the ess data frame to ess.csv

essv.to_csv(os.path.join(path, '2.2 Prepared data', 'essv.csv'))